In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
PATH = Path.cwd().parent.joinpath('data')

## Import data

In [ ]:
# Importeer data over gemeenten, opsplitsingen, samenvoegingen en CBS data over onderwijs in de regio

gemeenten = pd.read_csv(PATH.joinpath('gemeenten', 'gemeenten_per_jaar.csv'), dtype={'code': str})

opsplitsingen = pd.read_csv(PATH.joinpath('gemeenten', 'opsplitsingen.csv'), dtype={'code_nieuw': str,
                                                                                    'code_oud': str})
samenvoegingen = pd.read_csv(PATH.joinpath('gemeenten', 'samenvoegingen.csv'), dtype={'code_nieuw': str,
                                                                                    'code_oud': str})

onderwijs = pd.read_csv(PATH.joinpath('onderwijs', 'speciaal_onderwijs.csv'), sep=';')

print(f'Imported {len(gemeenten)} gemeenten, {len(opsplitsingen)} opsplitsingen and {len(samenvoegingen)} samenvoegingen')

In [ ]:
# Clean up onderwijs dataset

onderwijs.rename(columns={'Regiokenmerken': 'gemeente', 
                           'Leerlingen/studenten (aantal)': 'aantal_leerlingen',
                           'Perioden': 'jaar'}, inplace=True)

onderwijs.columns = onderwijs.columns.str.lower()
onderwijs = onderwijs[onderwijs.gemeente != 'Gemeenten; niet in te delen'].copy()
onderwijs.jaar = onderwijs.jaar.str[0:4].astype(int)
onderwijs.dropna(subset='aantal_leerlingen', inplace=True)
onderwijs.aantal_leerlingen = onderwijs.aantal_leerlingen.astype(int)

In [ ]:
# Filter and process the onderwijs dataframe for different onderwijssoort

def process_onderwijs(df, onderwijssoort, groupby_cols=['gemeente', 'jaar'], drop_cols=['geslacht', 'onderwijssoort']):
    filtered_df = df[df['onderwijssoort'].isin(onderwijssoort)].copy()
    if len(onderwijssoort) > 1:
        filtered_df = filtered_df.groupby(groupby_cols).sum().reset_index()
    filtered_df.drop(drop_cols, axis=1, inplace=True)
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

vso = process_onderwijs(onderwijs, ["Voortgezet speciaal ondewijs "])
vo = process_onderwijs(onderwijs, ["Totaal voortgezet onderwijs"])
pso = process_onderwijs(onderwijs, ["Speciaal onderwijs", "Speciaal basisonderwijs"])
po = process_onderwijs(onderwijs, ["Basisonderwijs"])

print(f'len(vso): {len(vso)}, len(vo): {len(vo)}, len(pso): {len(pso)}, len(po): {len(po)}')

In [ ]:
# Define the dictionary for replacements
to_replace = {
    'Gaasterl‚n-Sleat': 'Gaasterlân-Sleat',
    ' (gemeente)': '',
    "'s-Hertogenbosch": 'Den Bosch',
    's-Hertogenbosch': 'Den Bosch',
    "'s-Gravenhage": 'Den Haag',
    's-Gravenhage': 'Den Haag',
    'S˙dwest-Frysl‚n': 'Súdwest-Fryslân',
    'Skarsterl‚n': 'Skarsterlân',
    'Den Bosch/Oss': 'Den Bosch',
    'Groesbeek': 'Berg en Dal',
    'De Friese Meren': 'De Fryske Marren',
}

# List of dataframes and columns to apply replacements
dataframes = [
    (vso, 'gemeente'),
    (vo, 'gemeente'),
    (pso, 'gemeente'),
    (po, 'gemeente'),
    (gemeenten, 'gemeentenaam'),
    (samenvoegingen, 'gemeentenaam_nieuw'),
    (samenvoegingen, 'gemeentenaam_oud'),
    (opsplitsingen, 'gemeente_naam_nieuw'),
    (opsplitsingen, 'gemeentenaam_oud')
]

# Apply replacements
for df, column in dataframes:
    for key, value in to_replace.items():
        df[column] = df[column].str.replace(key, value).str.strip()

In [ ]:
# Clean a bit more

for df in [vso, vo, pso, po]:
    df = pd.merge(df, gemeenten, left_on=['jaar', 'gemeente'], right_on=['jaar', 'gemeentenaam'], how='left')
    df.drop('gemeentenaam', axis=1, inplace=True)
    df = df[df.gemeente != 'Gemeenten; niet in te delen'].copy()
    df = df.sort_values(['jaar', 'gemeente'], ascending=False)
    df = df[df.gemeente != 'Gemeenten; niet in te delen'].copy()


In [ ]:
# Verwerk herindelingen

def update_municipality_names(df, samenvoegingen, opsplitsingen):
    # Merge with samenvoegingen to get the new municipality names
    df_merged = pd.merge(df, samenvoegingen[['gemeentenaam_oud', 'gemeentenaam_nieuw']],
                         left_on='gemeente', right_on='gemeentenaam_oud', how='left')

    # Update the municipality names based on the merged information
    df_merged['new_gemeente'] = df_merged['gemeentenaam_nieuw'].fillna(df_merged['gemeente'])

    # Drop unnecessary columns
    df_merged.drop(['gemeentenaam_nieuw', 'gemeentenaam_oud'], axis=1, inplace=True)

    # Handle opsplitsingen
    for _, row in opsplitsingen.iterrows():
        mask = df_merged['gemeente'] == row['gemeentenaam_oud']
        if mask.any():
            split_count = opsplitsingen[opsplitsingen['gemeentenaam_oud'] == row['gemeentenaam_oud']].shape[0]
            df_merged.loc[mask, 'aantal_leerlingen'] = df_merged.loc[mask, 'aantal_leerlingen'] / split_count
            df_merged.loc[mask, 'new_gemeente'] = row['gemeente_naam_nieuw']

    return df_merged

# Update municipality names for each dataframe
vso_updated = update_municipality_names(vso, samenvoegingen, opsplitsingen)
vo_updated = update_municipality_names(vo, samenvoegingen, opsplitsingen)
pso_updated = update_municipality_names(pso, samenvoegingen, opsplitsingen)
po_updated = update_municipality_names(po, samenvoegingen, opsplitsingen)

# Group by year and new municipality name and sum the number of students
vso_grouped = vso_updated.groupby(['jaar', 'new_gemeente']).agg({'aantal_leerlingen': 'sum'}).reset_index()
vo_grouped = vo_updated.groupby(['jaar', 'new_gemeente']).agg({'aantal_leerlingen': 'sum'}).reset_index()
pso_grouped = pso_updated.groupby(['jaar', 'new_gemeente']).agg({'aantal_leerlingen': 'sum'}).reset_index()
po_grouped = po_updated.groupby(['jaar', 'new_gemeente']).agg({'aantal_leerlingen': 'sum'}).reset_index()

vso_grouped['dataset'] = 'vso'
vo_grouped['dataset'] = 'vo'
pso_grouped['dataset'] = 'pso'
po_grouped['dataset'] = 'po'

# Combine all grouped dataframes
primary = pd.concat([pso_grouped, po_grouped]).reset_index(drop=True)
secondary = pd.concat([vso_grouped, vo_grouped]).reset_index(drop=True)

for df in [primary, secondary]:
    df = df.groupby(['jaar', 'new_gemeente', 'dataset']).agg({'aantal_leerlingen': 'sum'}).reset_index()
    df.aantal_leerlingen = df.aantal_leerlingen.astype(int)
print(f'Unique municipality names in primary: {primary.new_gemeente.nunique()} and secondary: {secondary.new_gemeente.nunique()}')

In [ ]:
# Filter the primary dataframe for the years 2013 and 2023

primary_2013 = primary[(primary['jaar'] == 2013) & (primary['dataset'] == 'po')]
primary_2023 = primary[(primary['jaar'] == 2023) & (primary['dataset'] == 'po')]
pso_2013 = primary[(primary['jaar'] == 2013) & (primary['dataset'] == 'pso')]
pso_2023 = primary[(primary['jaar'] == 2023) & (primary['dataset'] == 'pso')]

# Merge the dataframes on new_gemeente to get the required columns
merged_primary = primary_2013[['new_gemeente', 'aantal_leerlingen']].merge(
    primary_2023[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente', suffixes=('_2013', '_2023')
).merge(
    pso_2013[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente'
).merge(
    pso_2023[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente', suffixes=('_so_2013', '_so_2023')
)

# Calculate the required columns
merged_primary['perc_difference_so'] = round(
    ((merged_primary['aantal_leerlingen_so_2023'] - merged_primary['aantal_leerlingen_so_2013']) / merged_primary['aantal_leerlingen_so_2013']) * 100, 2
)
merged_primary['perc_so_2013'] = round(
    (merged_primary['aantal_leerlingen_so_2013'] / merged_primary['aantal_leerlingen_2013']) * 100, 2
)
merged_primary['perc_so_2023'] = round(
    (merged_primary['aantal_leerlingen_so_2023'] / merged_primary['aantal_leerlingen_2023']) * 100, 2
)
merged_primary['difference_perc_points'] = round(
    merged_primary['perc_so_2023'] - merged_primary['perc_so_2013'], 2)

# Convert aantal_leerlingen columns to integers
for col in ['aantal_leerlingen_2013', 'aantal_leerlingen_2023', 'aantal_leerlingen_so_2013', 'aantal_leerlingen_so_2023']:
    merged_primary[col] = merged_primary[col].astype(int)

primary = merged_primary


In [ ]:

# Filter the secondary dataframe for the years 2013 and 2023
secondary_2013 = secondary[(secondary['jaar'] == 2013) & (secondary['dataset'] == 'vo')]
secondary_2023 = secondary[(secondary['jaar'] == 2023) & (secondary['dataset'] == 'vo')]
vso_2013 = secondary[(secondary['jaar'] == 2013) & (secondary['dataset'] == 'vso')]
vso_2023 = secondary[(secondary['jaar'] == 2023) & (secondary['dataset'] == 'vso')]

# Merge the dataframes on new_gemeente to get the required columns
merged_secondary = secondary_2013[['new_gemeente', 'aantal_leerlingen']].merge(
    secondary_2023[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente', suffixes=('_2013', '_2023')
).merge(
    vso_2013[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente'
).merge(
    vso_2023[['new_gemeente', 'aantal_leerlingen']], on='new_gemeente', suffixes=('_vso_2013', '_vso_2023')
)

# Calculate the required columns
merged_secondary['perc_difference_so'] = round(
    ((merged_secondary['aantal_leerlingen_vso_2023'] - merged_secondary['aantal_leerlingen_vso_2013']) / merged_secondary['aantal_leerlingen_vso_2013']) * 100, 2
)
merged_secondary['perc_vso_2013'] = round(
    (merged_secondary['aantal_leerlingen_vso_2013'] / merged_secondary['aantal_leerlingen_2013']) * 100, 2
)
merged_secondary['perc_vso_2023'] = round(
    (merged_secondary['aantal_leerlingen_vso_2023'] / merged_secondary['aantal_leerlingen_2023']) * 100, 2
)
merged_secondary['difference_perc_points'] = round(
    merged_secondary['perc_vso_2023'] - merged_secondary['perc_vso_2013'], 2)

# Convert aantal_leerlingen columns to integers
for col in ['aantal_leerlingen_2013', 'aantal_leerlingen_2023', 'aantal_leerlingen_vso_2013', 'aantal_leerlingen_vso_2023']:
    merged_secondary[col] = merged_secondary[col].astype(int)


secondary = merged_secondary


In [ ]:
# Write to file

primary.to_csv(PATH.joinpath('primair_onderwijs.csv'), index=False)
secondary.to_csv(PATH.joinpath('secundair_onderwijs.csv'), index=False)

### Some extra stuff

In [ ]:
metric = 'difference_perc_points'

# Count positive and negative values in merged_secondary
positive_secondary = (secondary[metric] > 0).sum()
negative_secondary = (secondary[metric] < 0).sum()

# Count positive and negative values in merged_primary
positive_primary = (primary[metric] > 0).sum()
negative_primary = (primary[metric] < 0).sum()

print(f'Positive verschil_2013_2023 in merged_secondary: {positive_secondary}')
print(f'Negative verschil_2013_2023 in merged_secondary: {negative_secondary}')
print(f'Positive verschil_2013_2023 in merged_primary: {positive_primary}')
print(f'Negative verschil_2013_2023 in merged_primary: {negative_primary}')

In [ ]:
# Calculate percentiles for secondary dataframe
secondary['perc_so_2013_percentile'] = secondary['perc_vso_2013'].rank(pct=True)
secondary['perc_so_2023_percentile'] = secondary['perc_vso_2023'].rank(pct=True)

# Calculate percentiles for primary dataframe
primary['perc_so_2013_percentile'] = primary['perc_so_2013'].rank(pct=True)
primary['perc_so_2023_percentile'] = primary['perc_so_2023'].rank(pct=True)

# Calculate the difference in percentiles
secondary['difference_percentiles'] = secondary['perc_so_2023_percentile'] - secondary['perc_so_2013_percentile']
primary['difference_percentiles'] = primary['perc_so_2023_percentile'] - primary['perc_so_2013_percentile']